In [1]:
from models import ChatMessage, ChatHistory
import dspy
from lms.together import Together

from modules.chatter import ChatterModule

C:\Users\subho\anaconda3\envs\whisper2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
lm = Together(
    model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
    temperature=0.5,
    max_tokens=1000,
    top_p=0.7,
    top_k=50,
    repetition_penalty=1.2,
    stop=["<|eot_id|>", "<|eom_id|>", "\n\n---\n\n", "\n\n---", "---", "\n---"],
    # stop=["\n", "\n\n"],
)

In [3]:
dspy.settings.configure(lm=lm)

In [5]:
import json
import random
from modules.chatter import (
    ChatMessage, 
    ChatHistory, 
    MessageContext,
    format_chat_history,
    format_context
)
import dspy

def load_and_split_data(file_path='../training_data/conversation_with_context.json'):
    # Load the dataset
    with open(file_path, 'r') as f:
        conversations = json.load(f)

    # Set random seed for reproducibility
    random.seed(42)
    random.shuffle(conversations)

    # Split into train (80%) and dev (20%) sets
    split_idx = int(len(conversations) * 0.8)
    train_data = conversations[:split_idx]
    dev_data = conversations[split_idx:]

    def create_example(conv):
        # Create MessageContext objects
        messages = []
        for msg in conv['chat_history']['messages']:
            context = MessageContext(**msg['context'])
            messages.append(ChatMessage(
                from_creator=msg['from_creator'],
                content=msg['content'],
                context=context
            ))
        
        # Create ChatHistory object
        chat_history = ChatHistory(messages=messages)
        last_context = messages[-1].context
        
        # Convert to strings for DSPy
        chat_str = format_chat_history(chat_history)
        context_str = format_context(last_context)
        
        return dspy.Example({
            'chat_history': chat_str,
            'context': context_str,
            'output': conv['output']
        }).with_inputs('chat_history', 'context')

    trainset = [create_example(conv) for conv in train_data]
    devset = [create_example(conv) for conv in dev_data]

    print(f"Training examples: {len(trainset)}")
    print(f"Dev examples: {len(devset)}")
    
    return trainset, devset
    
trainset, devset=load_and_split_data()

print(f"Training examples: {len(trainset)}")
print(f"Dev examples: {len(devset)}")

Training examples: 8
Dev examples: 2
Training examples: 8
Dev examples: 2


In [7]:
# Example from trainset
example = trainset[0]
print("Training Example Structure:")
print("Chat History:")
print(example.chat_history)  # Now this is a formatted string
print("\nContext:")
print(example.context)
print("\nExpected Output:")
print(example.output)

Training Example Structure:
Chat History:
Fan [Time: afternoon, Message #1]: I feel like I'm always 'rediscovering' myself. Is that normal, or am I just indecisive?
Creator [Time: afternoon, Message #2]: Rediscovery is a sign of growth. We're constantly evolving, and it's healthy to question who you are.
Fan [Time: afternoon, Message #3]: It feels chaotic, but I also feel like I'm learning so much about myself.

Context:
Current context: Time of day: afternoon, Message #3, Minutes since start: 10, Minutes since last: 5

Expected Output:
That's the beauty of self-discovery. Embrace the chaosâ€”it means you're alive and curious.


In [9]:
from dspy.teleprompt import KNNFewShot
from modules.chatter import ChatterModule

def metric(example, prediction, trace=None):
    # For string comparison
    pred_output = prediction if isinstance(prediction, str) else prediction.output
    expected_output = example.output if hasattr(example, 'output') else example
    return pred_output == expected_output

def test_knn(k_value, trainset):
    print(f"\nTesting with k={k_value}")
    print("-" * 50)
    
    bootstrap_args = {
        'metric': metric,
        'max_bootstrapped_demos': 2,
        'max_labeled_demos': 4,
        'max_rounds': 1,
        'max_errors': 5
    }

    knn_teleprompter = KNNFewShot(
        k=k_value,
        trainset=trainset,
        **bootstrap_args
    )

    chatter = ChatterModule(examples=None)
    compiled_knn = knn_teleprompter.compile(
        student=chatter,
        trainset=trainset
    )

    # Test with sample input
    test_history = trainset[0].chat_history
    test_context = trainset[0].context
    
    try:
        result = compiled_knn(chat_history=test_history, context=test_context)
        response = result if isinstance(result, str) else result.output
    except Exception as e:
        print(f"Error generating response: {e}")
        response = "Error generating response"
    
    try:
        # Get and print neighbors
        neighbors = knn_teleprompter.KNN(chat_history=test_history, context=test_context)
    except Exception as e:
        print(f"Error getting neighbors: {e}")
        neighbors = []
    
    print(f"\nInput conversation:")
    print(test_history)
    
    print(f"\nContext:")
    print(test_context)
    
    print(f"\nModel output: {response}")
    
    if neighbors:
        print(f"\nNearest {k_value} neighbors used:")
        for i, n in enumerate(neighbors):
            print(f"\nNeighbor {i+1}:")
            print("Chat History:")
            print(n.chat_history)
            print("Context:")
            print(n.context)
            print(f"Response: {n.output if hasattr(n, 'output') else 'No output available'}")

# Test with different k values
for k in [2, 3, 4]:
    test_knn(k, trainset)


Testing with k=2
--------------------------------------------------


100%|█████████████████████████████████████████████████████| 2/2 [00:09<00:00,  4.77s/it]


Bootstrapped 0 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.

Input conversation:
Fan [Time: afternoon, Message #1]: I feel like I'm always 'rediscovering' myself. Is that normal, or am I just indecisive?
Creator [Time: afternoon, Message #2]: Rediscovery is a sign of growth. We're constantly evolving, and it's healthy to question who you are.
Fan [Time: afternoon, Message #3]: It feels chaotic, but I also feel like I'm learning so much about myself.

Context:
Current context: Time of day: afternoon, Message #3, Minutes since start: 10, Minutes since last: 5

Model output: This journey of discovery will help shape your values and passions over time. Enjoy the process!

Nearest 2 neighbors used:

Neighbor 1:
Chat History:
Fan [Time: afternoon, Message #1]: I feel like I'm always 'rediscovering' myself. Is that normal, or am I just indecisive?
Creator [Time: afternoon, Message #2]: Rediscovery is a sign of growth. We're constantly evolving, and it's healthy to

100%|█████████████████████████████████████████████████████| 3/3 [00:18<00:00,  6.07s/it]


Bootstrapped 0 full traces after 2 examples for up to 1 rounds, amounting to 3 attempts.

Input conversation:
Fan [Time: afternoon, Message #1]: I feel like I'm always 'rediscovering' myself. Is that normal, or am I just indecisive?
Creator [Time: afternoon, Message #2]: Rediscovery is a sign of growth. We're constantly evolving, and it's healthy to question who you are.
Fan [Time: afternoon, Message #3]: It feels chaotic, but I also feel like I'm learning so much about myself.

Context:
Current context: Time of day: afternoon, Message #3, Minutes since start: 10, Minutes since last: 5

Model output: Remember, every step forward might mean leaving some things behind, but what you gain from each journey inward will enrich your path ahead.

Nearest 3 neighbors used:

Neighbor 1:
Chat History:
Fan [Time: afternoon, Message #1]: I feel like I'm always 'rediscovering' myself. Is that normal, or am I just indecisive?
Creator [Time: afternoon, Message #2]: Rediscovery is a sign of growth. We'

100%|█████████████████████████████████████████████████████| 4/4 [00:21<00:00,  5.44s/it]


Bootstrapped 0 full traces after 3 examples for up to 1 rounds, amounting to 4 attempts.

Input conversation:
Fan [Time: afternoon, Message #1]: I feel like I'm always 'rediscovering' myself. Is that normal, or am I just indecisive?
Creator [Time: afternoon, Message #2]: Rediscovery is a sign of growth. We're constantly evolving, and it's healthy to question who you are.
Fan [Time: afternoon, Message #3]: It feels chaotic, but I also feel like I'm learning so much about myself.

Context:
Current context: Time of day: afternoon, Message #3, Minutes since start: 10, Minutes since last: 5

Model output: That's the beauty of self-discovery. Embrace the chaos—it means you're alive and curious.

Nearest 4 neighbors used:

Neighbor 1:
Chat History:
Fan [Time: afternoon, Message #1]: I feel like I'm always 'rediscovering' myself. Is that normal, or am I just indecisive?
Creator [Time: afternoon, Message #2]: Rediscovery is a sign of growth. We're constantly evolving, and it's healthy to questi

In [13]:
# First cell - imports
from rouge_score import rouge_scorer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sentence_transformers import SentenceTransformer
import random
from dspy.teleprompt import KNNFewShot
from modules.chatter import ChatterModule

# Second cell - evaluator class
class ResponseEvaluator:
    def __init__(self):
        self.scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        self.encoder = SentenceTransformer('all-MiniLM-L6-v2')
        
    def calculate_metrics(self, prediction, reference):
        # ROUGE scores
        try:
            scores = self.scorer.score(reference, prediction)
            rouge_l = scores['rougeL'].fmeasure
        except:
            rouge_l = 0
            
        # Semantic similarity using SBERT
        pred_emb = self.encoder.encode([prediction])[0]
        ref_emb = self.encoder.encode([reference])[0]
        semantic_sim = cosine_similarity([pred_emb], [ref_emb])[0][0]
        
        return {
            'rouge_l': rouge_l,
            'semantic_sim': semantic_sim
        }

# Third cell - test function
def test_knn(k_value, trainset, test_example, verbose=True):
    bootstrap_args = {
        'metric': metric,
        'max_bootstrapped_demos': 2,
        'max_labeled_demos': 4,
        'max_rounds': 1,
        'max_errors': 5
    }

    knn_teleprompter = KNNFewShot(
        k=k_value,
        trainset=trainset,
        **bootstrap_args
    )

    chatter = ChatterModule(examples=None)
    compiled_knn = knn_teleprompter.compile(
        student=chatter,
        trainset=trainset
    )

    try:
        result = compiled_knn(
            chat_history=test_example.chat_history,
            context=test_example.context
        )
        response = result if isinstance(result, str) else result.output
    except Exception as e:
        if verbose:
            print(f"Error generating response: {e}")
        response = "Error generating response"
    
    return response

# Fourth cell - evaluation function
def evaluate_k_values(trainset, k_values=[2, 3, 4], num_samples=5):  # Reduced samples for faster testing
    evaluator = ResponseEvaluator()
    results = {k: {'rouge': [], 'semantic': []} for k in k_values}
    
    # Sample conversations to test
    test_indices = random.sample(range(len(trainset)), min(num_samples, len(trainset)))
    
    for k in k_values:
        print(f"\nTesting k={k} with {num_samples} samples")
        print("-" * 50)
        
        for idx in test_indices:
            test_example = trainset[idx]
            
            # Get model response for this k value
            response = test_knn(k, trainset, test_example, verbose=False)
            
            # Calculate metrics
            metrics = evaluator.calculate_metrics(response, test_example.output)
            
            results[k]['rouge'].append(metrics['rouge_l'])
            results[k]['semantic'].append(metrics['semantic_sim'])
            
            print(f"\nSample {idx + 1}:")
            messages = test_example.chat_history.split('\n')
            print(f"Input: {messages[-1]}")  # Last message
            print(f"Expected: {test_example.output}")
            print(f"Generated: {response}")
            print(f"ROUGE-L: {metrics['rouge_l']:.3f}")
            print(f"Semantic Similarity: {metrics['semantic_sim']:.3f}")
    
    # Calculate averages
    print("\nOverall Results:")
    print("-" * 50)
    for k in k_values:
        avg_rouge = np.mean(results[k]['rouge'])
        avg_semantic = np.mean(results[k]['semantic'])
        std_rouge = np.std(results[k]['rouge'])
        std_semantic = np.std(results[k]['semantic'])
        
        print(f"\nk={k}:")
        print(f"Average ROUGE-L: {avg_rouge:.3f} (±{std_rouge:.3f})")
        print(f"Average Semantic Similarity: {avg_semantic:.3f} (±{std_semantic:.3f})")
    
    return results

# Fifth cell - run evaluation
# Make sure trainset is already loaded
results = evaluate_k_values(trainset, k_values=[2, 3, 4], num_samples=5)  # Reduced samples for faster testing



Testing k=2 with 5 samples
--------------------------------------------------


100%|█████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.36s/it]


Bootstrapped 0 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.

Sample 1:
Input: Fan [Time: afternoon, Message #3]: It feels chaotic, but I also feel like I'm learning so much about myself.
Expected: That's the beauty of self-discovery. Embrace the chaosâ€”it means you're alive and curious.
Generated: You're on a journey of continuous discovery, which takes courage and openness. Keep exploring!
ROUGE-L: 0.188
Semantic Similarity: 0.578


100%|█████████████████████████████████████████████████████| 2/2 [00:09<00:00,  4.73s/it]


Bootstrapped 0 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.

Sample 5:
Input: Fan [Time: evening, Message #3]: I love that idea. Maybe right now, I'm in my 'mystical science' chapter.
Expected: Exactly! Allow yourself to dive deeply into each chapter. Your interests are like seasonsâ€”they come and go with purpose.
Generated: Exactly! Allow yourself to dive deeply into each chapter. Your interests are like seasons—they come and go with purpose.
ROUGE-L: 1.000
Semantic Similarity: 0.981


...

Bootstrapped 0 full traces after 3 examples for up to 1 rounds, amounting to 4 attempts.

Sample 2:
Input: Fan [Time: morning, Message #3]: Wow, I never thought of it that way. Like, each key is a choice or a mystery.
Expected: Exactly! Sometimes the things we keep have deeper symbolism, even if we don't realize it consciously.
Generated: "It seems like your collection holds not only aesthetic appeal but also emotional depth. Have you ever considered what draws you specifically to one piece over another?"
ROUGE-L: 0.045
Semantic Similarity: 0.314


100%|█████████████████████████████████████████████████████| 4/4 [00:21<00:00,  5.35s/it]


Bootstrapped 0 full traces after 3 examples for up to 1 rounds, amounting to 4 attempts.

Sample 7:
Input: Fan [Time: morning, Message #3]: I'll try focusing on my small joysâ€”like vintage records and herbal teas.
Expected: Perfect. Those are your anchors. In a chaotic world, even small rituals can be deeply meaningful.
Generated: Perfect. Those are your anchors. In a chaotic world, even small rituals can be deeply meaningful.
ROUGE-L: 1.000
Semantic Similarity: 1.000


100%|█████████████████████████████████████████████████████| 4/4 [00:21<00:00,  5.35s/it]


Bootstrapped 0 full traces after 3 examples for up to 1 rounds, amounting to 4 attempts.

Sample 4:
Input: Fan [Time: evening, Message #3]: That's really reassuring. Sometimes I just want to escape into another time.
Expected: That's understandable. Just remember that you can bring elements of those eras into your life now.
Generated: You might enjoy exploring historical reenactments or themed events as a way to step into other times without leaving yours behind.
ROUGE-L: 0.158
Semantic Similarity: 0.472

Overall Results:
--------------------------------------------------

k=2:
Average ROUGE-L: 0.269 (±0.371)
Average Semantic Similarity: 0.502 (±0.263)

k=3:
Average ROUGE-L: 0.680 (±0.393)
Average Semantic Similarity: 0.783 (±0.252)

k=4:
Average ROUGE-L: 0.535 (±0.404)
Average Semantic Similarity: 0.661 (±0.279)


In [14]:
from dspy.teleprompt import KNNFewShot
from modules.chatter import ChatterModule
import os

def metric(example, prediction, trace=None):
    return example.output == prediction.output

# Create the KNN teleprompter with optimal parameters
bootstrap_args = {
    'metric': metric,
    'max_bootstrapped_demos': 2,
    'max_labeled_demos': 4,
    'max_rounds': 1,
    'max_errors': 5
}

# Initialize KNN with k=3 (as per our evaluation)
knn_teleprompter = KNNFewShot(
    k=3,  # Using 3 nearest neighbors based on evaluation
    trainset=trainset,
    **bootstrap_args
)

# Create and compile
chatter = ChatterModule(examples=None)
compiled_knn = knn_teleprompter.compile(
    student=chatter,
    trainset=trainset
)

# Create directory if it doesn't exist
os.makedirs("saved_models", exist_ok=True)

# Save the compiled model
compiled_knn.save("saved_models/optimized_chatbot_with_context.json")

# Test function to verify the model works with context
def test_model():
    test_history = trainset[0].chat_history
    test_context = trainset[0].messages[-1].context  # Get context from last message
    
    result = compiled_knn(
        chat_history=test_history,
        context=test_context
    )
    
    print("\nTest Results:")
    print("Input conversation:")
    print(test_history)  # Using string representation
    
    print(f"\nContext:")
    print(f"Time of day: {test_context.time_of_day}")
    print(f"Message #: {test_context.message_number}")
    print(f"Minutes since start: {test_context.minutes_since_start}")
    print(f"Minutes since last: {test_context.minutes_since_last}")
    
    print(f"\nModel output: {result.output}")
    
    # Get and print neighbors with context
    neighbors = knn_teleprompter.KNN(
        chat_history=test_history,
        context=test_context
    )
    
    print(f"\nNearest {len(neighbors)} neighbors used:")
    for i, n in enumerate(neighbors):
        print(f"\nNeighbor {i+1}:")
        print("Chat History:")
        print(n.chat_history)
        print("Context:")
        print(n.context)
        print(f"Response: {n.output}")

# Uncomment to test
# test_model()

print(f"\nModel saved to: saved_models/optimized_chatbot_with_context.json")


Model saved to: saved_models/optimized_chatbot_with_context.json


In [16]:
from modules.chatter import ChatterModule, ChatHistory, ChatMessage, MessageContext
from datetime import datetime
import time

# Load the optimized model
chatter = ChatterModule(examples=None)
chatter.load(path="saved_models/optimized_chatbot_with_context.json")

# Initialize chat history
chat_history = ChatHistory(messages=[])
start_time = time.time()
last_message_time = start_time

def get_context(message_number):
    current_time = time.time()
    
    # Get time of day
    hour = datetime.now().hour
    if 5 <= hour < 12:
        time_of_day = "morning"
    elif 12 <= hour < 17:
        time_of_day = "afternoon"
    else:
        time_of_day = "evening"
    
    # Calculate times
    minutes_since_start = int((current_time - start_time) / 60)
    minutes_since_last = int((current_time - last_message_time) / 60)
    
    return MessageContext(
        message_number=message_number,
        time_of_day=time_of_day,
        minutes_since_start=minutes_since_start,
        minutes_since_last=minutes_since_last
    )

def format_chat_history(history):
    """Convert ChatHistory to formatted string"""
    result = []
    for msg in history.messages:
        speaker = "Creator" if msg.from_creator else "Fan"
        context = f"[Time: {msg.context.time_of_day}, Message #{msg.context.message_number}]"
        result.append(f"{speaker} {context}: {msg.content}")
    return "\n".join(result)

def format_context(context):
    """Convert MessageContext to formatted string"""
    return (f"Current context: Time of day: {context.time_of_day}, "
            f"Message #{context.message_number}, "
            f"Minutes since start: {context.minutes_since_start}, "
            f"Minutes since last: {context.minutes_since_last}")

print("Chat started! (Type 'quit' to exit)")
print("-" * 50)

message_counter = 0
while True:
    # Get user input
    user_input = input("You: ").strip()
    
    if user_input.lower() == 'quit':
        print("\nChat ended. Goodbye!")
        break
    
    # Update counter and get context
    message_counter += 1
    context = get_context(message_counter)
    
    # Append user input to chat history
    chat_history.messages.append(
        ChatMessage(
            from_creator=False,
            content=user_input,
            context=context
        )
    )

    try:
        # Format inputs as strings
        chat_str = format_chat_history(chat_history)
        context_str = format_context(context)
        
        # Get response using loaded model with context
        response = chatter(
            chat_history=chat_str,
            context=context_str
        )

        # Update last message time
        last_message_time = time.time()
        
        # Update counter and get new context for response
        message_counter += 1
        context = get_context(message_counter)
        
        # Append response to chat history
        chat_history.messages.append(
            ChatMessage(
                from_creator=True,
                content=response,
                context=context
            )
        )
        
        # Print response with some formatting
        print("\nCreator:", response)
        print("-" * 50)
        
    except Exception as e:
        print(f"\nError: {str(e)}")
        print("Please try again.")
        print("-" * 50)

Chat started! (Type 'quit' to exit)
--------------------------------------------------


You:  who are you?



Creator: Hello! I'm an assistant designed to help answer questions and provide information on various topics. How can I assist you today?
--------------------------------------------------


You:  what is your name?



Creator: While I don't have a personal name like a human does, I'm here to be helpful and assist with any questions or tasks you might have! Think of me as your go-to resource whenever you need something clarified or looked up. Is there something specific you'd like assistance with tonight?
--------------------------------------------------


KeyboardInterrupt: Interrupted by user